<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 41.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import pytorch_lightning as pl
#from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [3]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [6]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [7]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [8]:
train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")
validation = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/validation.csv")
test = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/boneage-test-dataset.csv")

In [9]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [10]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['boneage'] = train['boneage'].apply(lambda x: str(x))
train['male'] = train['male'].apply(lambda x: 0 if x else 1)

df1 = pd.DataFrame(train)
df1.rename(columns={'male':'gender'},inplace=True)

In [11]:
df1

,id,boneage,gender
0,1377.png,180,1
1,1378.png,12,1
2,1379.png,94,1
3,1380.png,120,0
4,1381.png,82,1
...,...,...,...
10084,13259.png,94,1
10085,13261.png,94,1
10086,13274.png,94,1
10087,13286.png,94,1


In [12]:
train

,id,boneage,gender
0,1377.png,180,1
1,1378.png,12,1
2,1379.png,94,1
3,1380.png,120,0
4,1381.png,82,1
...,...,...,...
10084,13259.png,94,1
10085,13261.png,94,1
10086,13274.png,94,1
10087,13286.png,94,1


In [13]:
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
validation['boneage'] = validation['boneage'].apply(lambda x: str(x))
validation['male'] = validation['male'].apply(lambda x: 0 if x else 1)

df2 = pd.DataFrame(validation)
df2.rename(columns={'male':'gender'},inplace=True)

In [14]:
validation

,id,boneage,gender
0,10186.png,10,0
1,10758.png,15,1
2,10860.png,180,1
3,10877.png,180,1
4,11008.png,180,1
...,...,...,...
2517,15605.png,50,1
2518,15606.png,113,1
2519,15608.png,55,1
2520,15609.png,150,0


In [15]:
test

,Case ID,Sex,Ground truth bone age (months)
0,4360,M,169
1,4361,M,170
2,4362,M,73
3,4363,M,153
4,4364,M,135
...,...,...,...
195,4555,F,133
196,4556,F,129
197,4557,F,168
198,4558,F,136


In [16]:
test['Case ID'] = test['Case ID'].apply(lambda x: str(x)+'.png')
#test['boneage'] = test['boneage'].apply(lambda x: str(x))
#test['Sex'] = test['Sex'].apply(lambda x: 0 if x else 1)

df3 = pd.DataFrame(test)
df3.rename(columns={'Sex':'gender'},inplace=True)
df3['gender'].replace({ 'M':0,'F':1}, inplace=True)


In [17]:
test

,Case ID,gender,Ground truth bone age (months)
0,4360.png,0,169
1,4361.png,0,170
2,4362.png,0,73
3,4363.png,0,153
4,4364.png,0,135
...,...,...,...
195,4555.png,1,133
196,4556.png,1,129
197,4557.png,1,168
198,4558.png,1,136


#clone

##ลองดึง Inception_V3 จาก github ตัวเอง

In [18]:
#Local imports
#from Project_Boneage.backbone.inception_v3 import InceptionV3

##ลองดึง Inception_V3 จาก github ของ Bonet มาใช้ก่อน

In [19]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet


Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [20]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

#Creating a custom PyTorch Dataset

In [21]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, pickle_file, image_dir):
        self.image_dir = image_dir
        self.pickle_file = pickle_file
        self.tabular = pd.read_csv(pickle_file)

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        y = tabular["boneage"]

        image = Image.open(f"{self.image_dir}{tabular['id']}.png")
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)
        image = transforms.functional.resize(image, size=[500,500])
        # image = transforms.Compose([transforms.Resize((500, 500)),
        #                        transforms.RandomAffine(
        #                            20, translate=(0.2, 0.2),
        #                            scale=(1, 1.2)),
        #                        transforms.RandomHorizontalFlip(),
        #                        transforms.ToTensor()])

        tabular = tabular[["male"]]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return image, tabular, y

In [22]:
table=pd.read_csv(path+"/train.csv")
table

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [23]:
np.array(Image.open(f"{train_img}{table.iloc[0,0:]['id']}.png"))[..., :3].shape

(2044, 3)

In [24]:
ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")

In [25]:
#import torchvision.models as models
#inception = models.inception_v3(init_weights=True)

In [26]:
class BoNetModule(pl.LightningModule):

    def __init__(
        self, transform_input = False, lr = 1e-3, num_workers = 4, batch_size = 32
    ):

        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x, y):
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

    def training_step(self, batch, batch_idx):
      image, tabular, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, tabular))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)

      tensorboard_logs = {"train_loss": loss}
      return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

    def setup(self, stage):

        image_datatrain = ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")
        image_dataval = ImageDataset(pickle_file=path+"/validation.csv", image_dir=f"{val_img}")
        image_datatest = ImageDataset(pickle_file=path+"/boneage-test-dataset.csv", image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr))

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    

class DenseLayer(pl.LightningModule):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)

In [27]:
ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")[0][0].size()

<ipython-input-21-4a3b2084aa90>:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  tabular = torch.FloatTensor(tabular)


torch.Size([1, 500, 500])

#Training

In [28]:
from fastai.vision.learner import Learner
from fastai.callback.schedule import lr_find

In [29]:
logger = TensorBoardLogger("lightning_logs", name="image_only")
#early_stop_callback = EarlyStopping(monitor="loss",min_delta=5000,patience=7, verbose=False, mode="min")


In [30]:
model = BoNetModule()
#trainer = pl.Trainer(accelerator='cpu', logger=logger, callbacks=early_stop_callback)
#trainer = pl.Trainer(accelerator='cpu', gpus=1, logger=logger)#, early_stop_callback=early_stop_callback)
trainer = pl.Trainer(gpus=1, logger=logger)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# lr_finder = trainer.lr_find(model)
# fig = lr_finder.plot(suggest=True, show=True)
# new_lr = lr_finder.suggestion()
# print(new_lr)
# model.hparams.lr = new_lr

trainer.fit(model)
trainer.test(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type        | Params
---------------------------------------------
0 | inception_v3 | InceptionV3 | 21.8 M
1 | gender       | DenseLayer  | 64    
2 | fc_1         | DenseLayer  | 100 M 
3 | fc_2         | DenseLayer  | 1.0 M 
4 | fc_3         | Linear      | 1.0 K 
---------------------------------------------
123 M     Trainable params
0         Non-trainable params
123 M     Total params
492.688   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

<ipython-input-21-4a3b2084aa90>:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  tabular = torch.FloatTensor(tabular)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
tensorboard --logdir=lightning_logs